# Chatbot for HammerHead

## Install packages

In [1]:
%pip install langchain --quiet
# %pip install pypdf --quiet
%pip install openai --quiet
# %pip install unstructured --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# %pip install tiktoken --quiet

In [3]:
# %pip install "langchain[docarray]" -- quiet

In [4]:
# %pip install jq --quiet

In [5]:
# !pip install "docarray[hnswlib]" --quiet
!pip install faiss-cpu --quiet


In [6]:
!pip install voila --quiet

## Load the Vectorstore

In [14]:
from langchain.vectorstores import FAISS
# from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings.openai import OpenAIEmbeddings
import config


In [15]:
embeddings = OpenAIEmbeddings()


In [16]:
vectorstore = FAISS.load_local('./HammerHead/hh_faiss.store', embeddings=embeddings)

In [17]:
# product_doc = 'Lista de Produtos. Esta página apresenta todas as categorias de produtos que estão disponíveis na loja HammerHead.'
# product_doc += 'Os produtos estão divididos nas seguintes categorias:'
# product_doc += 'Roupas de natação, Acessórios de natação, raquetes de beach tennis.' 

# vectorstore.add_texts(texts=[product_doc])
# vectorstore.save_local('./HammerHead/hh_faiss.store')

## Creating RetrievalQA with Memory

In [18]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
import langchain

from langchain.memory import ConversationBufferMemory

In [19]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [20]:
qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(temperature=0), 
    retriever=vectorstore.as_retriever(top_k=10), 
    memory=memory,
    return_source_documents=False,
    verbose=False

)

In [21]:
# langchain.debug=False
# result = qa({"question": 'Qual marca você representa?'})
# langchain.debug=False
# result['answer']

In [22]:
# result = qa({"question": 'Quais categorias de produtos vocês possuem?'})
# result['answer']

In [23]:
# result = qa({"question": 'Quais óculos vocês possuem?'})
# result['answer']

In [24]:
# result = qa({"question": 'Quanto custa o Hammerhead Avenger?'})
# result['answer']

In [25]:
# result = qa({"question": 'Eu gostaria de comprar o Hammerhead Avenger?'})
# result['answer']

## Adding the Interactive Chat

In [ ]:
from datetime import datetime
from IPython.display import HTML, display
from ipywidgets import widgets

In [ ]:
%%html
<link rel="stylesheet" 
      href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" 
      integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" 
      crossorigin="anonymous">
<style>
    body{margin-top:20px;}

    .chat-message-left,
    .chat-message-right {
        display: flex;
        flex-shrink: 0
    }

    .chat-message-left {
        margin-right: auto
    }

    .chat-message-right {
        flex-direction: row-reverse;
        margin-left: auto
    }
</style>

In [ ]:

chat_history = []


def text_eventhandler(*args):
    # Needed bc when we "reset" the text input
    # it fires instantly another event since
    # we "changed" it's value to ""
    if args[0]["new"] == "":
        return

    # Show loading animation
    loading_bar.layout.display = "block"

    # Get question
    question = args[0]["new"]

    # Reset text field
    args[0]["owner"].value = ""

    # Formatting question for output
    q = (
        f'<div class="chat-message-right pb-4"><div>'
        + f'<img src="images/bear.png" class="rounded-circle mr-1" width="40" height="40">'
        + f'<div class="text-muted small text-nowrap mt-2">{datetime.now().strftime("%H:%M:%S")}</div></div>'
        + '<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">You</div>{question}</div>'
    )

    # Display formatted question
    output.append_display_data(HTML(q))

    try:
        response = qa({"question": f"{question}", "chat_history": chat_history})
        answer = response["answer"]
        chat_history.append((question, answer))
    except Exception as e:
        answer = "<b>Error:</b> " + str(e)

    # Formatting answer for output
    # Replacing all $ otherwise matjax would format them in a strange way
    answer_formatted = answer.replace('$', r'\$')
    a = (
        f'<div class="chat-message-left pb-4"><div>'
        + f'<img src="images/cat.png" class="rounded-circle mr-1" width="40" height="40">'
        + f'<div class="text-muted small text-nowrap mt-2">{datetime.now().strftime("%H:%M:%S")}</div></div>'
        + '<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">LLM</div>{answer_formatted}</div>'
    )

    # Turn off loading animation
    loading_bar.layout.display = "none"

    output.append_display_data(HTML(a))

In [ ]:
in_text = widgets.Text()
in_text.continuous_update = False
in_text.observe(text_eventhandler, "value")
output = widgets.Output()

file = open("./images/loading.gif", "rb")
image = file.read()
loading_bar = widgets.Image(
    value=image, format="gif", width="20", height="20", layout={"display": "None"}
)

In [30]:
display(
    widgets.HBox(
        [output],
        layout=widgets.Layout(
            width="100%",
            max_height="500px",
            display="inline-flex",
            flex_flow="column-reverse",
        ),
    )
)


display(
    widgets.Box(
        children=[loading_bar, in_text],
        layout=widgets.Layout(display="flex", flex_flow="row"),
    )
)

Box(children=(Image(value=b'GIF89a\xc8\x00\xc8\x00\xf7\x00\x00\xab\xbd\x81\xf8\xd6\xd8\xfa\xc5\x8f\xfc\xdf\xd7…

In [10]:
txt_input.value='Boa tarde'
btn_send.click()

In [3]:
%pip install panel 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.1.0
    Uninstalling bokeh-3.1.0:
      Successfully uninstalled bokeh-3.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import panel

ModuleNotFoundError: No module named 'panel'

In [2]:
import param

ModuleNotFoundError: No module named 'param'